In [3]:
import os, sys

# Example Notebook Code
nbook_dir = os.getcwd()

# Add the parent directory of the 'scripts' folder to sys.path
sys.path.append(os.path.abspath(os.path.join(nbook_dir, '..')))

In [5]:
from scripts.data_loader import load_stock
from scripts.data_analysis import calculate_moving_average
from scripts.data_visualization import plot_stock_data
from scripts.data_visualization import plot_univariate
from scripts.data_visualization import plot_bivariate


ImportError: cannot import name 'load_stock' from 'scripts.data_loader' (c:\Users\Alaa\Documents\KIFIYA\kaim-week-1\stock_price_analysis\scripts\data_loader.py)